In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#### Imports and TPU setting

In [2]:
! pip install --upgrade kaggle -q
! pip install transformers -q
! pip install emoji -qq
! pip install googletrans -qq

     |████████████████████████████████| 890kB 3.4MB/s 
     |████████████████████████████████| 890kB 16.6MB/s 
     |████████████████████████████████| 3.0MB 22.5MB/s 
     |████████████████████████████████| 1.1MB 45.4MB/s 
     |████████████████████████████████| 51kB 1.8MB/s 
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 952kB 10.0MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 102kB 6.4MB/s 


In [3]:
import os
import re
import time
import numpy as np
import pandas as pd
import transformers
from tqdm import tqdm
import tensorflow as tf
from google.colab import files
import tensorflow_datasets as tfds
from transformers import BertTokenizer
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from transformers import TFAutoModel, AutoTokenizer
from tensorflow.keras.layers import Dense, Input,Dropout


from setup import set_TPU
from text_models import XLMRobertaInputs

import matplotlib.pyplot as plt
%matplotlib inline

tf.get_logger().setLevel('ERROR')

#### Load the data

In [4]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c google-quest-challenge

Saving kaggle.json to kaggle.json
  0% 0.00/954k [00:00<?, ?B/s]
100% 954k/954k [00:00<00:00, 62.4MB/s]
  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 37.4MB/s]
  0% 0.00/4.50M [00:00<?, ?B/s]
100% 4.50M/4.50M [00:00<00:00, 74.0MB/s]


In [5]:
!unzip '/content/train.csv.zip'

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [6]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [7]:
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [8]:
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [9]:
print(f"train shape: {train.shape} \nvalidation shape: {test.shape}")

train shape: (6079, 41) 
validation shape: (476, 11)


#### Preprocess

In [10]:
output_categories = list(train.columns[11:])
input_categories = list(train.columns[[1,2,5]])

In [11]:
train['question_title_body']  = train['question_title'] + '. ' + train['question_body']

#### Modelling

###### Build model inputs

In [12]:
strategy = set_TPU()

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU: grpc://10.92.140.34:8470
REPLICAS: 8


In [13]:
# Configuration
MODEL = 'jplu/tf-xlm-roberta-large'
EPOCHS = 10
MAX_LEN = 96

BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [28]:
xlmroberta_inputs_train = XLMRobertaInputs(train[['question_title_body','answer']].iloc[:5000].values.tolist(), train[output_categories].iloc[:5000].values, max_length=MAX_LEN, batch_size=BATCH_SIZE)
%time train_inputs = xlmroberta_inputs_train.process_examples(train=True)

xlmroberta_inputs_val = XLMRobertaInputs(train[['question_title_body','answer']].iloc[5000:].values.tolist(), train[output_categories].iloc[5000:].values, max_length=MAX_LEN, batch_size=BATCH_SIZE)
%time validation_inputs = xlmroberta_inputs_val.process_examples(train=False)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 8.58 s, sys: 14.8 ms, total: 8.59 s
Wall time: 8.61 s


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 1.98 s, sys: 31.6 ms, total: 2.01 s
Wall time: 2.01 s


In [35]:
def build_model(transformer,max_len):
    
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    sequence_output = transformer(input_ids)[0]
    cls_token = sequence_output[:, 0, :]
    cls_token = Dropout(0.2)(cls_token)
    cls_token = Dense(512,activation='relu')(cls_token)
    out = Dense(len(output_categories), activation='softmax')(cls_token)

    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        Adam(lr=2e-5), 
        loss='binary_crossentropy'
    )
    
    return model

In [36]:
def scheduler(epoch, lr):
  if epoch < 3:
    return lr*1.1
  else:
    return lr * tf.math.exp(-0.1)

In [37]:
def scheduler(epoch, lr, lr_start=0.00001, lr_max=0.00003, lr_min=0.000001, lr_rampup_epochs=3, lr_sustain_epochs=0, lr_exp_decay=.6):
  if epoch < lr_rampup_epochs:
      lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
  elif epoch < lr_rampup_epochs + lr_sustain_epochs:
      lr = lr_max
  else:
      lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
  return lr

In [38]:
transformer_layer = TFAutoModel.from_pretrained(MODEL)
model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some weights of the model checkpoint at jplu/tf-xlm-roberta-large were not used when initializing TFXLMRobertaModel: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the model checkpoint at jplu/tf-xlm-roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 96)]              0         
_________________________________________________________________
tfxlm_roberta_model_2 (TFXLM ((None, 96, 1024), (None, 559890432 
_________________________________________________________________
tf_op_layer_strided_slice_2  [(None, 1024)]            0         
_________________________________________________________________
dropout_224 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 30)                15390     
Total params: 560,430,622
Trainable params: 560,430,622
Non-trainable params: 0
________________________________________

###### Training

In [39]:
n_steps = train.shape[0]//BATCH_SIZE
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

train_history = model.fit(train_inputs, validation_data=validation_inputs, steps_per_epoch=n_steps, callbacks=[callback], epochs=EPOCHS, verbose=1)

Epoch 1/10
47/47 [==============================] - 951s 20s/step - loss: 1.5436 - val_loss: 1.4828
Epoch 2/10
47/47 [==============================] - 949s 20s/step - loss: 1.4900 - val_loss: 1.4734
Epoch 3/10
47/47 [==============================] - 950s 20s/step - loss: 1.4753 - val_loss: 1.4635
Epoch 4/10
47/47 [==============================] - 950s 20s/step - loss: 1.4652 - val_loss: 1.4585
Epoch 5/10
47/47 [==============================] - 949s 20s/step - loss: 1.4598 - val_loss: 1.4544
Epoch 6/10
47/47 [==============================] - 949s 20s/step - loss: 1.4555 - val_loss: 1.4529
Epoch 7/10
47/47 [==============================] - 949s 20s/step - loss: 1.4520 - val_loss: 1.4526
Epoch 8/10
47/47 [==============================] - 950s 20s/step - loss: 1.4486 - val_loss: 1.4523
Epoch 9/10
47/47 [==============================] - 949s 20s/step - loss: 1.4494 - val_loss: 1.4521
Epoch 10/10
47/47 [==============================] - 952s 20s/step - loss: 1.4474 - val_loss: 1.4520

In [40]:
text = [['its a nice day', 'the weather outside seems to be good']]
token = xlmroberta_inputs_train.tokenizer.batch_encode_plus(text,
                        max_length = MAX_LEN, # max length of the text that can go to BERT
                        pad_to_max_length = True, # add [PAD] tokens
                        truncation = True
                    )
xlmroberta_inputs_pred = np.array(token['input_ids'])

model.predict(xlmroberta_inputs_pred) # Entailment

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


array([[0.05337222, 0.04186647, 0.01936486, 0.03515103, 0.03512163,
        0.03663982, 0.06049836, 0.04315512, 0.0150277 , 0.00426378,
        0.03354704, 0.01078019, 0.00522507, 0.00479721, 0.00243124,
        0.00583045, 0.01241555, 0.00642372, 0.06547349, 0.00227671,
        0.06081361, 0.07315592, 0.04936613, 0.06560604, 0.08499067,
        0.05417708, 0.02544255, 0.00897466, 0.03787101, 0.04594063]],
      dtype=float32)

###### Build model